# Yeast Reference for RiboFlow

We prepare references for riboflow for the yeast genome.
We already have the CDS, 5' UTR and 3' UTR sequences.
We merge these sequences to assemble a transcriptome for yeast.
We observed that there may be more than 1 UTR sequence for the same transcript. So we picked one UTR sequence randomly. In total we obtained ~5000 transcript sequences.

We also assume that the start codon belongs to CDS whereas the stop codon belongs to 3' UTR. So we adjusted the CDS ends by -3 in ther method `produce_region_entries`.

Sequences are obtained from [https://www.yeastgenome.org/](https://www.yeastgenome.org/) around March 14, 2022.

In [1]:
import sys
import os
from collections import defaultdict
import gzip

%matplotlib inline

import IPython

# Some formatting forthe rest of the notebook
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [2]:
sys.path.insert(0, './')
import ref_lib
from ref_lib.Fasta import FastaFile, FastaEntry
from ref_lib.GTF import GTFfile, GTFEntry, get_gtf_contents


In [3]:
cds_fasta_file = "../raw_sequences/orf_coding.fa.gz"
three_p_file   = "../raw_sequences/SGD_all_ORFs_3prime_UTRs.fa.gz" 
five_p_file    = "../raw_sequences/SGD_all_ORFs_5prime_UTRs.fa.gz"

In [4]:
cds_handle        = FastaFile(cds_fasta_file)
cds_sequence_dict = {e.header : e.sequence for e in cds_handle}

three_p_utr_handle = FastaFile(three_p_file)
three_p_dict_pre   = {e.header : e.sequence for e in three_p_utr_handle}

five_p_utr_handle = FastaFile(five_p_file)
five_p_dict_pre   = {e.header : e.sequence for e in five_p_utr_handle}


In [5]:
def get_utr_gene_name(x):
    return x.split("_")[4]

assert( get_utr_gene_name('sacCer3_ct_Pelechanoonlybased3primeUTRs_3950_YGL227W_id001_three_prime_UTR') == 'YGL227W' )

In [6]:
three_p_dict = dict( zip( map(get_utr_gene_name, three_p_dict_pre.keys()) , three_p_dict_pre.values() ) )
five_p_dict  = dict( zip( map(get_utr_gene_name, five_p_dict_pre.keys()), five_p_dict_pre.values() )) 

In [7]:
print("There are \n{} 5p UTR and\n{} 3p UTR sequences in the raw files.".format( len(five_p_dict_pre), len(three_p_dict_pre) ))

There are 
9723 5p UTR and
9723 3p UTR sequences in the raw files.


In [8]:
print("There are {} ORF\n{} 5p UTR and\n{} 3p UTR sequences.".format(len(cds_sequence_dict), len(five_p_dict), len(five_p_dict) ) )

There are 6034 ORF
5211 5p UTR and
5211 3p UTR sequences.


In [9]:
common_utr_genes = set(three_p_dict.keys()).intersection( set(five_p_dict.keys()) )
print("There are {} genes in the intersection of 3p and 5p sequence genes.".format(len(common_utr_genes)) )

There are 5211 genes in the intersection of 3p and 5p sequence genes.


In [10]:
common_cds_and_utr_genes = set(three_p_dict.keys()).intersection( set(cds_sequence_dict.keys()) )
print("{} is the number of genes in the intersection of CDS and UTR sequences.".format(len(common_cds_and_utr_genes)))

5024 is the number of genes in the intersection of CDS and UTR sequences.


The difference between the raw and processed headers is because somegenes have more than one UTR annotation. Below is an example:

In [11]:
three_p_dict_pre["sacCer3_ct_Pelechanoonlybased3primeUTRs_3950_YAL025C_id001_three_prime_UTR"]

'AGCTCACGCACTATACTACGCAACATTACATGATTTCAGTTTCTATTATTGAATGAACAGGATTTTTCCACGTGGATTGGGCAGCGCACTGTACCATAATAACAACCAGTTATAAAATATTTCAATTCTTTTACATATATAAAAATACATAAAGACTATTATTTTTGTAAGATGGCCCATTCTGTAACACTTAAATTTCTTATCAAAAAAAAAAAAAATAATACCTTTAGTAGCTGTTGGGCTAATAACATTGCAAATTAGAAGCTTACTATTAGAAGCTTACTATT'

In [12]:
three_p_dict_pre["sacCer3_ct_Pelechanoonlybased3primeUTRs_3950_YAL025C_id002_three_prime_UTR"]

'AGCTCACGCACTATACTACGCAACATTACATGATTTCAGTTTCTATTATTGAATGAACAGGATTTTTCCACGTGGATTGGGCAGCGCACTGTACCATAATAACAACCAGTTATAAAATATTTCAATTCTTTTACATATATAAAAATACATAAAGACTATTATTTTTGTAAGATGGCCCATTCTGTAACACTTAAATTTCTTATCAAAAAAAAAAAAAATAATACCTTTAGTAGCTGTTGGGCTAATAACATTGCAAATTAGAAGCTTACTATTAGAAGCTT'

In [13]:
def make_bed_unit_entry(gene, start, end, region):
    return "{gene}\t{start}\t{end}\t{region}\t0\t+".format(gene=gene, start = start, end = end, region = region)


def produce_region_entries(gene, fivep_sequence, cds_sequence, threep_sequence):
    
    five_p_start_position = 0
    five_p_end_position   = len(fivep_sequence)
    
    # Stop codon belongs to 3' UTR not the CDS
    # So we have -3 below.
    cds_start_position = five_p_end_position
    cds_end_position   = cds_start_position + len(cds_sequence) - 3
    
    threep_utr_start_position = cds_end_position
    threep_utr_end_position = threep_utr_start_position + len(threep_sequence) + 3
    
    five_p_entry = make_bed_unit_entry(gene, five_p_start_position, five_p_end_position, "UTR5")
    cds_entry    = make_bed_unit_entry(gene, cds_start_position, cds_end_position, "CDS")
    threep_entry = make_bed_unit_entry(gene, threep_utr_start_position, threep_utr_end_position, "UTR3")
    
    return five_p_entry + "\n" + cds_entry + "\n" + threep_entry


obtained_result = produce_region_entries("mygene", "AGGC", "AGTCCCTAG", "CCCC")

expected_result  = "mygene\t0\t4\tUTR5\t0\t+\n"
expected_result += "mygene\t4\t10\tCDS\t0\t+\n"
expected_result += "mygene\t10\t17\tUTR3\t0\t+"

print(obtained_result,"\n", expected_result)

assert(obtained_result == expected_result)

mygene	0	4	UTR5	0	+
mygene	4	10	CDS	0	+
mygene	10	17	UTR3	0	+ 
 mygene	0	4	UTR5	0	+
mygene	4	10	CDS	0	+
mygene	10	17	UTR3	0	+


In [14]:
regions_file       = "../reference/yeast_regions.bed"
transcritpome_file = "../reference/yeast_transcriptome.fa.gz"
transcript_lengths = "../reference/yeast_lengths.tsv"

with open(regions_file, "wt") as region_handle,\
     gzip.open(transcritpome_file ,"wt") as transcriptome_handle,\
     open(transcript_lengths, "wt") as length_handle:
    
    for g in cds_sequence_dict.keys():
        if g not in five_p_dict.keys():
            continue
        transcript_sequence = five_p_dict[g] + cds_sequence_dict[g] + three_p_dict[g]
        this_entry          = FastaEntry(g, transcript_sequence)
        
        print(this_entry , file = transcriptome_handle )
    
        transcript_len_entry = "{}\t{}".format( g, len(transcript_sequence) )
        print(transcript_len_entry, file = length_handle)
        
        
        region_entry = produce_region_entries(g, five_p_dict[g], cds_sequence_dict[g], three_p_dict[g])
        
        print(region_entry, file = region_handle)